# Проект "Анализ факторов, влияющих на успех продаж игр"

**Описание проекта**: Представим ситуацию, что я работаю в интернет-магазине «Стримчик», который продаёт по всему миру компьютерные игры. Из открытых источников доступны исторические данные о продажах игр, оценки пользователей и экспертов, жанры и платформы (например, Xbox или PlayStation). Мне нужно выявить определяющие успешность игры закономерности. Это позволит сделать ставку на потенциально популярный продукт и спланировать рекламные кампании.
Перед мной данные до 2016 года. Представим, что сейчас декабрь 2016 г., и я планирую кампанию на 2017-й. Нужно отработать принцип работы с данными. Неважно, прогнозируете ли я продажи на 2017 год по данным 2016-го или же 2027-й — по данным 2026 года.
   
В наборе данных попадается аббревиатура ESRB (Entertainment Software Rating Board) — это ассоциация, определяющая возрастной рейтинг компьютерных игр. ESRB оценивает игровой контент и присваивает ему подходящую возрастную категорию, например, «Для взрослых», «Для детей младшего возраста» или «Для подростков».

**Цель исследования**: Выявить определяющие успешность игры закономерности, которые позволят сделать ставку на потенциально популярный продукт и спланировать рекламные кампании.

**Ход исследования**: 
- Подготовка данных: загрузка и изучение общей информации из предоставленного датасета;
- Предобработка данных: обработка пропущенных значений, корректировка типа данных, дубликатов и других аномалий;
- Расширение набора данных: добавление нового столбца - суммарные продажи во всех регионах;
- Исследовательский анализ данных: 
    - посмотреть, сколько игр выпускалось в разные годы;
    - посмотреть, как менялись продажи по платформам;
    - определить какие платформы лидируют по продажам, растут или падают;
    - построить график «ящик с усами» по глобальным продажам игр в разбивке по платформам;
    - посмотреть, как влияют на продажи внутри одной популярной платформы отзывы пользователей и критиков;
    - посмотреть на общее распределение игр по жанрам.
- Составление портрета пользователя каждого региона;
- Проверка гипотез.

**Описание данных**:
- Name — название игры
- Platform — платформа
- Year_of_Release — год выпуска
- Genre — жанр игры
- NA_sales — продажи в Северной Америке (миллионы проданных копий)
- EU_sales — продажи в Европе (миллионы проданных копий)
- JP_sales — продажи в Японии (миллионы проданных копий)
- Other_sales — продажи в других странах (миллионы проданных копий)
- Critic_Score — оценка критиков (максимум 100)
- User_Score — оценка пользователей (максимум 10)
- Rating — рейтинг от организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.

Данные за 2016 год могут быть неполными.

**Общий вывод**: Резюмирование полученных результатов, формулировка ключевых выводов и рекомендаций.

In [ ]:
# импорт библиотек
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as st

## Загрузка данных

In [ ]:
# считаем CSV-файл с данными с помощью библиотеки pandas и сохраним в датафрейм
data_games = pd.read_csv('/Users/numberone/Documents/Projects/datasets/games.csv')

In [ ]:
# выведем первые 10 cтрок таблицы data_games
display(data_games.head(10))

In [ ]:
# посмотрим общую информацию о каждом датафрейме
data_games.info()

In [ ]:
# посмотрим сводную статистику всех числовых столбцов
data_games.describe()

**Вывод**

Видим в данных data_games 16715 строк и 11 столбцов, вес датафрейма - 1,4 MB; столбцы Name, Platform, Genre, User_Score и Rating имеют тип данных - object; столбцы Year_of_Release, NA_sales, EU_sales, JP_sales, Other_sales и Critic_Score имеют тип данных - float64.
   
Необходимо привести названия столбцов к нижнему регистру.

Изменить тип данных столбцов Year_of_Release и Critic_Score на int64 (т.к. значения должны быть целочисленными), столбец User_Score на float64.

В столбцах содержатся пропуски, обработаем их в следующем разделе.

## Предобработка данных

### Заменим названия столбцов (приведем к нижнему регистру)

In [ ]:
data_games.columns = data_games.columns.str.lower()
display(data_games.head())

### Проверим наличие пропущенных значений и дубликатов в датафрейме и обработаем

In [ ]:
# посчитаем количество пропущенных значений в датафрейме
print(f'Количество пропущенных значений в data_games: {data_games.isna().sum().sum()}')

# посчитаем количество явных дубликатов в датафрейме
print(f'Количество явных дубликатов в data_games: {data_games.duplicated().sum()}')

In [ ]:
# посмотрим на неявные дубликаты
data_games[data_games[['name', 'platform','year_of_release']].duplicated(keep=False)]

Видим две похожие строки под номером 659 и 14244, эти данные не подлежат восстановление, поэтому можно их удалить.

Строку 604 с неявным дубликатом удалять нельзя, т.к. она содержит данные по продажам в Северной Америке и соответственно строку 16230 тоже нельзя удалить, т.к. она содержит данные о продажах в Европе. Объединим эти две строки.

In [ ]:
# удалим строки 659 и 14244
data_games.drop(index=[659, 14244], axis=0, inplace=True)

In [ ]:
# cложим значения 'eu_sales' в строках с неявными дубликатами
pd.options.mode.chained_assignment = None
data_games.eu_sales[604] = data_games.eu_sales[604] + data_games.eu_sales[16230]

# удалим строку дубликата
data_games.drop(index=[16230], axis=0, inplace=True)

# проверим
data_games.loc[
    (data_games['name'] == 'Madden NFL 13') & 
    (data_games['platform'] == 'PS3') & 
    (data_games['year_of_release'] == 2012)
]

In [ ]:
data_games.isna().sum()

In [ ]:
# посмотрим долю пропусков для дальнейшей обработки 
pd.DataFrame(round(data_games.isna().mean()*100,1)).style.background_gradient('coolwarm')

#### Столбец name

In [ ]:
# видим, что в стоблце name 2 пропуска, удалим их, удаление такого малого количества никак не повлияет на исследование
data_games = data_games.dropna(subset = ['name'])
data_games.isna().sum()

#### Столбец year_of_release

In [ ]:
# посчитаем процент пропусков в столбце year_of_release от всего датафрейма
'{:.1%}'.format((data_games[data_games['year_of_release'].isna()].shape[0]/data_games['year_of_release'].shape[0]))

In [ ]:
# удалим строки с пропусками в столбце year_of_release, будет потерено 1.6% строк с данными, что не является кричным
data_games = data_games.dropna(subset = ['year_of_release'])
data_games.isna().sum()

#### Столбец critic_score

In [ ]:
# посчитаем процент пропусков в столбце critic_score от всего датафрейма
'{:.1%}'.format((data_games[data_games['critic_score'].isna()].shape[0]/data_games['critic_score'].shape[0]))

Видим большое количество пропусков (возможно вследствие технического фактора или человеческого - оценки просто не были проставлены), поэтому будет неправильным удаление их. Каждая оценка индивидуальна и ее не получится заменить средним или медианным значением, поэтому оставим пропуски в данном столбце и заменим их 0.

In [ ]:
data_games['critic_score'] = data_games['critic_score'].fillna('0')
data_games['critic_score']

#### Столбец user_score 

In [ ]:
# посчитаем процент пропусков в столбце user_score от всего датафрейма
'{:.1%}'.format((data_games[data_games['user_score'].isna()].shape[0]/data_games['user_score'].shape[0]))

Видим большое количество пропусков (возможно вследствие технического фактора или человеческого - оценки пользователей просто не были проставлены), поэтому будет неправильным удаление их. Каждая оценка индивидуальна и ее не получится заменить средним или медианным значением, поэтому оставим пропуски в данном столбце и заменим их 0. Также видно, что пропуски в столбце сritic_score часто совпадают с пропусками в столбце user_score.

In [ ]:
data_games['user_score'] = data_games['user_score'].fillna('0')
data_games['user_score']

In [ ]:
# посмотрим на количество значений tbd в столбце user-score
# изучив информацию в интернете было выяснено, что значение tbd - это to be determined (предстоит определить)
data_games.query('user_score == "tbd"').shape

In [ ]:
# удалить данные значения нельзя, поэтому оставим их в данном столбце и заменим их 0.
data_games.loc[data_games['user_score'] == 'tbd', 'user_score'] = 0
data_games['user_score'].unique()

#### Столбец  rating

In [ ]:
# посчитаем процент пропусков в столбце user_score от всего датафрейма
'{:.1%}'.format((data_games[data_games['rating'].isna()].shape[0]/data_games['rating'].shape[0]))

Как и в предыдущих пунктах видим большое количество пропусков (возможно вследствие технического фактора или человеческого - рейтинги возрастных категорий просто не были проставлены), поэтому будет неправильным удаление их. Оставим пропуски в данном столбце и заменим их 0. Также видно, что пропуски в столбце rating часто совпадают с пропусками в столбцах сritic_score и user_score.

In [ ]:
data_games['rating'] = data_games['rating'].fillna('0')
data_games['rating']

### Преобразуем данные в нужные типы

In [ ]:
data_games['year_of_release'] = data_games['year_of_release'].astype('int')
data_games['critic_score'] = data_games['critic_score'].astype('int')
data_games['user_score'] = data_games['user_score'].astype('float')

# проверим
data_games.info()

### Суммарные продажи

In [ ]:
data_games['total_sales'] = (data_games['na_sales'] + data_games['eu_sales'] + data_games['jp_sales'] + data_games['other_sales'])
data_games.head()

**Вывод**

Были приведены названия столбцов к нижнему регистру.

Обработали пропуски в столбцах name, year_of_release, critic_score, user_score и rating. Отсутствие данных — это информация о том, что ни рейтинговое агентство ESRB, ни эксперты и пользователи не захотели или не смогли оценить эти игры. Либо как вариант, что какой-то технический фактор причастен к такому большому количеству пропусков.

Изменили тип данных столбцов Year_of_Release и Critic_Score на int64, столбец User_Score на float64.

Добавили новый столбец total_sales - суммарные продажи во всех регионах.

## Исследовательский анализ данных

### Cколько игр выпускалось в разные годы

In [ ]:
# посмотрим описательную статистику
data_games['year_of_release'].describe()

In [ ]:
# построим гистограмму
data_games.pivot_table(index='year_of_release', values='name', aggfunc='count').plot(kind='bar', 
                        grid=True, figsize=(15, 10))
plt.title('Динамика выпуска игр по годам')
plt.xlabel('Год')
plt.ylabel('Количество игр')
plt.show()

Согласно гистограмме видно, что количество продаваемых игр было малым до 1991 года, далее начался подъем продаж, самый пик продаж - это 2008 - 2009 года, далее произошел спад, но все равно выпускалось достатчно большое количество игр вплоть до 2016 года. Предполагаю, что такой спад связан со стремительным развитием цифровой индустрии - вследствии чего появились мобильные телефоны, общение в социальных сетях и мессенджерах, игры в мобильных телефонах и поэтому комппьютерные игры ушли на второй план. Думаю, что данные важны за все периоды.

### Как менялись продажи по платформам

In [ ]:
# составим таблицу с общей суммой продаж по каждой платформе
data_total_sales = pd.pivot_table(data_games, index=['platform'], values='total_sales', aggfunc='sum')
data_total_sales = data_total_sales.sort_values('total_sales',ascending=False)
data_total_sales

Видим, что максимальные продажи присуще платформам - PS2, X360, PS3, Wii, DS, PS. Построим графики распределение по годам для этих платформ.

In [ ]:
# построим гистограммы распределения суммарных продаж топа платформ
for platform in data_total_sales.index[:6]:
    (data_games
     .query('platform == @platform')
     .pivot_table(index='year_of_release', values='total_sales', aggfunc='sum')
     .plot(kind='bar', grid=True, figsize=(8, 5))
    )
    plt.title(platform)
    plt.xlabel('Год')
    plt.ylabel('Cуммарные продажи')
    plt.show()

PS2: видим продажи с 2000 по 2010 года

Х360: видим продажи с 2005 по 2016 года

PS3: видим продажи с 2006 по 2016 года
Wii: видим продажи с 2006 по 2015 года

DS: видим продажи с 2004 по 2013 года

PS: видим продажи с 1994 по 2003 года

В среднем можно сказать, что платформы существуют 10 лет; также видно, что платформы набирают максимальную популярность в среднем около 4 лет, далее идет пик продаж около 3 лет и соответственно спад.

### Актуальные данные

 Думаю, что актуальным периодом для исследования можно считать предшествующие 2017 году - пять лет, получается, что будем рассматривать данные за 2012, 2013, 2014, 2015, 2016 года (года ранее 2012 не берем в учет). Этих данных будет достаточно, чтобы построить прогноз на 2017 год.

### Какие платформы лидируют по продажам, растут или падают? 

In [ ]:
actual_period = data_games[data_games['year_of_release'] >= 2012]
actual_period

In [ ]:
# составим таблицу с общей суммой продаж по каждой платформе c 2012 по 2016 года
data_actual_period = pd.pivot_table(actual_period, index=['platform'], values='total_sales', aggfunc='sum')
data_actual_period = data_actual_period.sort_values('total_sales',ascending=False)
data_actual_period

Видим, что максимальные продажи за период 2012-2016 годов присуще платформам - PS4, PS3, X360, 3DS, XOne. Этот список отличается от списка за весь период продаж. Построим графики распределение по годам для этих платформ.

In [ ]:
# построим гистограммы распределения суммарных продаж топа платформ за период c 2012 по 2016 годов
for platform in data_actual_period.index:
    (actual_period
     .query('platform == @platform')
     .pivot_table(index='year_of_release', values='total_sales', aggfunc='sum')
     .plot(kind='bar', grid=True, figsize=(7, 5))
    )
    plt.title(platform)
    plt.xlabel('Год')
    plt.ylabel('Cуммарные продажи')
    plt.show()

Видим, что максимальные продажи за период 2012-2016 годов присуще платформам - PS4, PS3, X360, 3DS, XOne. Самые низкие продажи - у PSP.

Достаточное падение по продажам видим по платформам - PS3, X360, 3DS, PC, PSV, Wii, DS и PSP. 

Потенциально прибыльными платформами можно считать - PS4, Wii, XOne.

### График «ящик с усами» по глобальным продажам игр в разбивке по платформам

In [ ]:
# построим ящик с усами по глобальным продажам игр по платформам за актуальный период
actual_period.boxplot(by='platform', column=['total_sales'], figsize=(12,10), patch_artist=True, 
                      boxprops=dict(facecolor='r', color='b'))
plt.ylim(0,3)
plt.title('Продажи игр по платформам')
plt.xlabel('Платформа')
plt.ylabel('Суммарные продажи')
plt.show()

Согласно графика, видим, что все платформы имеют выбросы, максимальные продажи у платформы - X360, продажи у платформ поменьше, но рядом с Х360 - PS4, Wii и ХOne. Самые низкие продажи у платформ - PSP, PSV.

### Как влияют на продажи внутри одной популярной платформы отзывы пользователей и критиков

In [ ]:
# посмотрим на примере PS4 - самая популярная платформа
# и отфильтруем нулевые выбросы
actual_period = actual_period.query('user_score > 0 and critic_score > 0 and total_sales > 0')
sns.set_palette("dark")
sns.pairplot(actual_period[actual_period.platform == 'PS4'][['total_sales', 'critic_score', 'user_score']]) 
plt.show()

In [ ]:
# видим нулевые выбросы, это пропуски, которые были заменены на 0, используем корреляцию Спирмена, чтобы отбросить их
corr_1 = (actual_period[actual_period.platform == "PS4"][['total_sales', 'critic_score', 'user_score']].corr(method='spearman'))
corr_1.style.background_gradient(cmap='coolwarm')

На сильную положительную корреляцию указывает корреляция между user_score и critic_score - 0,797199, остальные связи являются слабыми.

### Соотношение вывода с продажами игр на других платформах

In [ ]:
# посмотрим на таблицу корреляции для платформы PS3
corr_2 = (actual_period[actual_period.platform == "PS3"][['total_sales', 'critic_score', 'user_score']].corr(method='spearman'))
corr_2.style.background_gradient(cmap='coolwarm')

In [ ]:
# построим диаграммы рессеяния для PS3
# и отфильтруем нулевые выбросы
actual_period = actual_period.query('user_score > 0 and critic_score > 0 and total_sales > 0')
sns.set_palette("dark")
sns.pairplot(actual_period[actual_period.platform == 'PS3'][['total_sales', 'critic_score', 'user_score']]) 
plt.show()

In [ ]:
# посмотрим на таблицу корреляции для платформы X360
corr_3 = (actual_period[actual_period.platform == "X360"][['total_sales', 'critic_score', 'user_score']].corr(method='spearman'))
corr_3.style.background_gradient(cmap='coolwarm')

In [ ]:
# построим диаграммы рессеяния для Х360
# и отфильтруем нулевые выбросы
actual_period = actual_period.query('user_score > 0 and critic_score > 0 and total_sales > 0')
sns.set_palette("dark")
sns.pairplot(actual_period[actual_period.platform == 'X360'][['total_sales', 'critic_score', 'user_score']]) 
plt.show()

In [ ]:
# посмотрим на таблицу корреляции для платформы 3DS
corr_4 = (actual_period[actual_period.platform == "3DS"][['total_sales', 'critic_score', 'user_score']].corr(method='spearman'))
corr_4.style.background_gradient(cmap='coolwarm')

In [ ]:
# построим диаграммы рессеяния для 3DS
# и отфильтруем нулевые выбросы
actual_period = actual_period.query('user_score > 0 and critic_score > 0 and total_sales > 0')
sns.set_palette("dark")
sns.pairplot(actual_period[actual_period.platform == '3DS'][['total_sales', 'critic_score', 'user_score']]) 
plt.show()

In [ ]:
# посмотрим на таблицу корреляции для платформы Xone
corr_5 = (actual_period[actual_period.platform == "XOne"][['total_sales', 'critic_score', 'user_score']].corr(method='spearman'))
corr_5.style.background_gradient(cmap='coolwarm')

In [ ]:
# построим диаграммы рессеяния для XOne
# и отфильтруем нулевые выбросы
actual_period = actual_period.query('user_score > 0 and critic_score > 0 and total_sales > 0')
sns.set_palette("dark")
sns.pairplot(actual_period[actual_period.platform == 'XOne'][['total_sales', 'critic_score', 'user_score']]) 
plt.show()

Самую сильную положительную зависимость можно увидеть для платформы 3DS между user_score и critic_score - 0,907987. Для остальных платформ все зависимости намного слабее.

### Общее распределение игр по жанрам

In [ ]:
# посмотрим суммарные продажи по жанрам за актуальный период
actual_period.pivot_table(index='genre', values='total_sales', aggfunc='median').sort_values('total_sales', 
                                                                                             ascending=False)

In [ ]:
# построим график распреления суммарных продаж относительно жанра за актуальный период
actual_period.pivot_table(index='genre', values='total_sales', aggfunc='sum').sort_values('total_sales', 
                                    ascending=False).plot(kind='bar', y='total_sales', figsize=(15, 7), legend=False)
plt.title('Продажи игр в зависимости от жанра')
plt.xlabel('Жанр')
plt.ylabel('Cуммарные продажи')
plt.show()

In [ ]:
# построим ящик с усами по глобальным продажам игр по жанрам за актуальный период
actual_period.boxplot(by='genre', column=['total_sales'], figsize=(20,15), patch_artist=True, 
                      boxprops=dict(facecolor='r', color='b'))
plt.ylim(0,4)
plt.title('Продажи игр в зависимости от жанра')
plt.xlabel('Жанр')
plt.ylabel('Cуммарные продажи')
plt.show()

С 2012-2016 года в топ-жанров входят - Schooter, Sports и Platform (жанр Shooter значительно выделяется по суммарным продажам относительно других жанров игр). На последнем месте - жанр Puzzle.

По большинству жанров присутствует большое количество выбросов.

**Вывод**

Cогласно исследовательскому анализу было выявлено:
- Пик продаж игр - это 2008 - 2009 года, далее был спад, но все равно выпускалось достаточно большое количество игр вплоть до 2016. 
- Максимальные продажи были у платформ - PS2, X360, PS3, Wii, DS, PS. В среднем платформы существуют 10 лет. Максимальную популярность набирают втечении 4 лет, далее идет пик продаж около 3 лет и соответственно спад.
- Актуальным периодом для дальнейшего исследования взяли 2012-2016 года. Максимальные продажи за этот период присуще платформам - PS4, PS3, X360, 3DS, XOne. Этот список отличается от списка за весь период продаж. Самые низкие продажи - у PSP. Потенциально прибыльными платформами можно считать - PS4, Wii, XOne.
- Сильная положительная корреляция между user_score и critic_score у платформ PS4 и 3DS.
- Самыми популярными жанрами за актуальный период являются - Schooter, Sports и Platform. На последнем месте - жанр Puzzle.

## Портрет пользователя каждого региона

### Анализ данных по популярным платформам, популярным жанрам и рейтингу ESRB в регионе NA

In [ ]:
# cохраним в переменную рейтинг продаж по Северной Америке для каждой платформы
platform_na = pd.pivot_table(actual_period, index='platform', values='na_sales',
                                        aggfunc='sum').sort_values('na_sales', ascending=False)
platform_na 

In [ ]:
# построим диаграмму для топ-5 платформ по продажам в Северной Америке
plt.figure(figsize=(10,7))
platform_na_graph = platform_na[:5].reset_index()
plt.pie(platform_na_graph['na_sales'],labels=platform_na_graph['platform'], autopct='%1.1f%%')
plt.axis('equal')
plt.show()

In [ ]:
# cохраним в переменную рейтинг продаж по Северной Америке для каждого жанра
genre_na = pd.pivot_table(actual_period, index='genre', values='na_sales',
                                        aggfunc='sum').sort_values('na_sales', ascending=False)
genre_na

In [ ]:
# построим диаграмму для топ-5 жанров по продажам в Северной Америке
plt.figure(figsize=(10,7))
genre_na_graph = genre_na[:5].reset_index()
plt.pie(genre_na_graph['na_sales'],labels=genre_na_graph['genre'], autopct='%1.1f%%')
plt.axis('equal')
plt.show()

In [ ]:
# cохраним в переменную рейтинг продаж по Северной Америке для каждого рейтинга
rating_na = pd.pivot_table(actual_period, index='rating', values='na_sales',
                                        aggfunc='sum').sort_values('na_sales', ascending=False)
rating_na

In [ ]:
# построим диаграмму для топ-5 рейтингов по продажам в Северной Америке
plt.figure(figsize=(9,6))
rating_na_graph = rating_na.reset_index()
plt.pie(rating_na_graph['na_sales'],labels=rating_na_graph['rating'], autopct='%1.1f%%')
plt.axis('equal')
plt.show()

Топ-5 платформ по продажам в Северной Америке - X360, PS4, PS3, XOne и 3DS.
Топ-5 жанров по продажам в Северной Америке - Action, Shooter, Sports, Role-Playing	и Misc.
Самым популярным рейтингом по продажам является - М.

### Анализ данных по популярным платформам, популярным жанрам и рейтингу ESRB в регионе EU

In [ ]:
# cохраним в переменную рейтинг продаж по Европе для каждой платформы
platform_eu = pd.pivot_table(actual_period, index='platform', values='eu_sales',
                                        aggfunc='sum').sort_values('eu_sales', ascending=False)
platform_eu 

In [ ]:
# построим диаграмму для топ-5 платформ по продажам в Европе
plt.figure(figsize=(10,7))
platform_eu_graph = platform_eu[:5].reset_index()
plt.pie(platform_eu_graph['eu_sales'],labels=platform_eu_graph['platform'], autopct='%1.1f%%')
plt.axis('equal')
plt.show()

In [ ]:
# cохраним в переменную рейтинг продаж по Европе для каждого жанра
genre_eu = pd.pivot_table(actual_period, index='genre', values='eu_sales',
                                        aggfunc='sum').sort_values('eu_sales', ascending=False)
genre_eu

In [ ]:
# построим диаграмму для топ-5 жанров по продажам в Европе
plt.figure(figsize=(10,7))
genre_eu_graph = genre_eu[:5].reset_index()
plt.pie(genre_eu_graph['eu_sales'],labels=genre_eu_graph['genre'], autopct='%1.1f%%')
plt.axis('equal')
plt.show()

In [ ]:
# cохраним в переменную рейтинг продаж по Европе для каждого рейтинга
rating_eu = pd.pivot_table(actual_period, index='rating', values='eu_sales',
                                        aggfunc='sum').sort_values('eu_sales', ascending=False)
rating_eu

In [ ]:
# построим диаграмму для топ-5 рейтингов по продажам в Европе
plt.figure(figsize=(9,6))
rating_eu_graph = rating_eu.reset_index()
plt.pie(rating_eu_graph['eu_sales'],labels=rating_eu_graph['rating'], autopct='%1.1f%%')
plt.axis('equal')
plt.show()

Топ-5 платформ по продажам в Европе - PS4, PS3, X360, XOne и 3DS. Топ-5 жанров по продажам в Европе - Action, Shooter, Sports, Role-Playing и Misc. Самым популярным рейтингом по продажам является - М.

### Анализ данных по популярным платформам, популярным жанрам и рейтингу ESRB в регионе JP

In [ ]:
# cохраним в переменную рейтинг продаж по Японии для каждой платформы
platform_jp = pd.pivot_table(actual_period, index='platform', values='jp_sales',
                                        aggfunc='sum').sort_values('jp_sales', ascending=False)
platform_jp 

In [ ]:
# построим диаграмму для топ-5 платформ по продажам в Японии
plt.figure(figsize=(10,7))
platform_jp_graph = platform_jp[:5].reset_index()
plt.pie(platform_jp_graph['jp_sales'],labels=platform_jp_graph['platform'], autopct='%1.1f%%')
plt.axis('equal')
plt.show()

In [ ]:
# cохраним в переменную рейтинг продаж по Японии для каждого жанра
genre_jp = pd.pivot_table(actual_period, index='genre', values='jp_sales',
                                        aggfunc='sum').sort_values('jp_sales', ascending=False)
genre_jp

In [ ]:
# построим диаграмму для топ-5 жанров по продажам в Японии
plt.figure(figsize=(10,7))
genre_jp_graph = genre_jp[:5].reset_index()
plt.pie(genre_jp_graph['jp_sales'],labels=genre_jp_graph['genre'], autopct='%1.1f%%')
plt.axis('equal')
plt.show()

In [ ]:
# cохраним в переменную рейтинг продаж по Японии для каждого рейтинга
rating_jp = pd.pivot_table(actual_period, index='rating', values='jp_sales',
                                        aggfunc='sum').sort_values('jp_sales', ascending=False)
rating_jp

In [ ]:
# построим диаграмму для топ-5 рейтингов по продажам в Японии
plt.figure(figsize=(9,6))
rating_jp_graph = rating_jp.reset_index()
plt.pie(rating_jp_graph['jp_sales'],labels=rating_jp_graph['rating'], autopct='%1.1f%%')
plt.axis('equal')
plt.show()

Топ-5 платформ по продажам в Японии - 3DS, PS3, PSV,PS4 и Wii. Топ-5 жанров по продажам в Японии - Role-Playing, Action, Misc, Simulation и Fighting. Самый популярный рейтинг по продажам не определен, второй по популярности - Е.

**Вывод**

Исходя из полученных данных, выяснили, что в Северной Америке предпочитают игры на платформе Х360 жанра Action с рейтингом М. В Европе игры на платформе PS4 жанра Action с рейтингом М. В Японии же на платформе 3DS жанра Role-Playing с рейтингом Е.

## Проверка гипотез

### Средние пользовательские рейтинги платформ Xbox One и PC одинаковые

Гипотеза Н0: Средние пользовательские рейтинги платформ Xbox One и PC одинаковые.

Гипотеза Н1: Средние пользовательские рейтинги платформ Xbox One и PC не одинаковые.

In [ ]:
alpha = 0.05
results = st.ttest_ind(
    actual_period.loc[actual_period['platform'] == 'XOne', 'user_score'].dropna(),
    actual_period.loc[actual_period['platform'] == 'PC', 'user_score'].dropna(),
    equal_var=False,
    alternative='two-sided'
)

print(results.pvalue)
if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу')
else:
    print('Не получилось отвергнуть нулевую минуту')

### Средние пользовательские рейтинги жанров Action и Sports разные

Гипотеза Н0: Средние пользовательские рейтинги жанров Action и Sports одинаковые.

Гипотеза Н1: Средние пользовательские рейтинги платформ Action и Sports не одинаковые.

In [ ]:
alpha = 0.05
results = st.ttest_ind(
    actual_period.loc[actual_period['genre'] == 'Action', 'user_score'].dropna(),
    actual_period.loc[actual_period['genre'] == 'Sports', 'user_score'].dropna(),
    equal_var=False,
    alternative='two-sided'
)

print(results.pvalue)
if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу')
else:
    print('Не получилось отвергнуть нулевую минуту')

**Вывод**

Первая гипотеза о том, что средние пользовательские рейтинги платформ Xbox One и PC одинаковые - подтвердилась. Это означает, что пользователи ставят примерно одинаковые оценки играм на плаформах Xbox One и PC.

Во второй гипотезе задание не удовлетворяет условия для составления нулевой гипотезы, т.к. сравнимаемые величины не равны. Для решения поменяли условия, если отвергнем нулевую гипотезу, значит подтвердим, что рейтинги разные. 
Было определено, что средние пользовательские рейтинги жанров Action и Sports разные.

## Общий вывод

1. Цель проекта - выявить определяющие успешность игры закономерности, которые позволят сделать ставку на потенциально популярный продукт и спланировать рекламные кампании.


2. В пункте Загрузка и предобработка данных было выполнено:
- привели названия столбцов к нижнему регистру;
- изменили тип данных столбцов Year_of_Release и Critic_Score на int64, столбец User_Score на float64;
- обработали пропуски в столбцах name, year_of_release, critic_score, user_score и rating. Отсутствие данных — это информация о том, что ни рейтинговое агентство ESRB, ни эксперты и пользователи не захотели или не смогли оценить эти игры. Либо как вариант, что какой-то технический фактор причастен к такому большому количеству пропусков;
- добавили новый столбец total_sales - суммарные продажи во всех регионах.


3. Cогласно исследовательскому анализу было выявлено:
- пик продаж игр - это 2008 - 2009 года, далее был спад, но все равно выпускалось достаточно большое количество игр вплоть до 2016;
- максимальные продажи были у платформ - PS2, X360, PS3, Wii, DS, PS. В среднем платформы существуют 10 лет. Максимальную популярность набирают втечении 4 лет, далее идет пик продаж около 3 лет и соответственно спад;
- актуальным периодом для дальнейшего был взят 2012-2016 года. Максимальные продажи за этот период присуще платформам - PS4, PS3, X360, 3DS, XOne. Этот список отличается от списка за весь период продаж. Самые низкие продажи - у PSP. Потенциально прибыльными платформами можно считать - PS4, Wii, XOne.
- Сильная положительная корреляция между user_score и critic_score у платформ PS4 и 3DS. Самыми популярными жанрами за актуальный период являются - Schooter, Sports и Platform. На последнем месте - жанр Puzzle.
- были построены графики распределения, графики "ящик с усами" и матрицы корреляции, благодаря которым были сделаны выводы выше.


4. Проанализировав данные, в том числе при помощи визуализации ввиде диаграмм составили портрет пользвателя для каждого региона: в Северной Америке предпочитают игры на платформе Х360 жанра Action с рейтингом М. В Европе игры на платформе PS4 жанра Action с рейтингом М. В Японии же на платформе 3DS жанра Role-Playing с рейтингом Е.


5. И подтвердили гипотезы о том, что средние пользовательские рейтинги платформ Xbox One и PC одинаковые и ,что средние пользовательские рейтинги платформ Action и Sports разные.


6. Рекомендации для рекламных кампаний: рекламные кампании должны быть направлены на игры платформы Х360 жанра Action с рейтингом М для пользователей Северной Америки, для европейских пользвателей на игры для  платформы PS4 жанра Action с рейтингом М и для японских пользвателей на игры для платформы 3DS жанра Role-Playing с рейтингом Е.

Спасибо за внимание!